In [1]:
from datasets import load_from_disk
import datasets

Concatenate datasets

In [2]:
data_folders = ["./annotations/annotations_bjurn_b1_10", "./annotations/annotations_bjurn_b2_10"]
data = load_from_disk(data_folders[0])
for folder in data_folders[1:]:
    data = datasets.concatenate_datasets([data, load_from_disk(folder)])


In [3]:
print(f'columns: {data.column_names}')
print(f'nrows: {data.num_rows}')

columns: ['doc_id', 'seg_id', 'translation', 'context']
nrows: 20


Context is stored as list of sentences, concatenate to a single, long string.

In [4]:
def concatenate_context(row, languages=None):
    if languages is None:
        languages = ['en', 'nl']
    row['context_concatenated'] = {
        lan: ' '.join(row['context'][lan]) for lan in languages
    }
    return row

In [5]:
data = data.map(concatenate_context)

In [6]:
for row in data:
    print(row['context_concatenated']['en'])
    print(row['context_concatenated']['nl'])
    print(row['translation']['en'])
    print(row['translation']['nl'])
    print()

And so on. There is a little bit of a difference there that is worth just --  -- thinking about. And I remember when I was 12 years old, and I started playing tympani and percussion, and my teacher said, "Well, how are we going to do this? You know, music is about listening." And I said, "Yes, I agree with that. So what's the problem?" And he said, "Well, how are you going to hear this? How are you going to hear that?"
en zo verder. Er zit een klein verschil op, dat de moeite is --  -- om over na te denken Ik herinner mij dat ik op mijn twaalfde op pauken en percussie begon te spelen. Mijn leraar zei: "Wel, hoe gaan we dat doen? Muziek gaat om luisteren." "Ja, akkoord. En wat is het probleem?" Hij zei: "Hoe ga je dit horen? Hoe ga je dat horen?"
And I said, "Well, how do you hear it?"
Ik zei: "Hoe hoor jij het?"

We truly do all understand it, and we have this global digital fireplace, don't we, but I want to share you with the world, because you are also a tribe. You are the TED tribe

In [7]:
# inseq seems to only work when doing 'pip install -r requirements.txt'
import inseq
from inseq.commands.attribute_context.attribute_context import attribute_context_with_model, AttributeContextArgs

Load inseq with mbart large en to nl

In [8]:
inseq_model = inseq.load_model(
    "facebook/mbart-large-50-one-to-many-mmt",
    "saliency",
    tokenizer_kwargs={'src_lang': 'en_XX', 'tgt_lang': 'nl_XX'},
)

In [9]:
def get_pecore_args(
        input_context_text,
        input_current_text,
        output_context_text,
        output_current_text=None,
        sample_identifier=None,
        output_folder='output'

):
    return AttributeContextArgs(
        model_name_or_path="facebook/mbart-large-50-one-to-many-mmt",
        attribution_method="saliency",
        attributed_fn="contrast_prob_diff",
        context_sensitivity_metric="kl_divergence",
        context_sensitivity_std_threshold=0,
        attribution_std_threshold=2,
        attribution_topk=5,
        input_context_text=input_context_text,
        input_current_text=input_current_text,
        output_context_text=output_context_text,
        # output_current_text=output_current_text,
        contextless_input_current_text="""{current}""",
        input_template="""{context} {current}""",
        contextless_output_current_text="""{current}""",
        output_template="{context} {current}",
        save_path=f"{output_folder}/{sample_identifier}.json",
        viz_path=f"{output_folder}/{sample_identifier}.html",
        tokenizer_kwargs={'src_lang': 'en_XX', 'tgt_lang': 'nl_XX'},
    )

Run little test

In [14]:
pec_args = get_pecore_args("This is context", "for translating this text", "Dit is context", sample_identifier="sample")

In [16]:
out = attribute_context_with_model(pec_args, inseq_model)

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  This is context
Input current: for translating this text
Output context: Dit is context
Output current: om deze tekst te vertalen.

#1.
Generated output (CTI > 0.346):  om(2.163) deze tekst te vertalen.
Input context (CCI > 0.099):     This is context
Output context (CCI > 0.102):    Dit is context

Now for use with the dataset

In [17]:
def get_pecore_args_for_row(row, sample_identifier=None, output_folder='output'):
    return get_pecore_args(
        row['context_concatenated']['en'],
        row['translation']['en'],
        row['context_concatenated']['nl'],
        sample_identifier=sample_identifier,
        output_folder=output_folder
    )

In [20]:
for i, row in enumerate(data):
    try:
        pecore_args = get_pecore_args_for_row(row, f"{i}_nl")
        out = attribute_context_with_model(pecore_args, inseq_model)
    except ValueError as e:
        print()
        print(f"ERROR for {i}")
        print(e)
        print()
        


ERROR for 0
Output template '{context} {current}' contains prefix 'en zo verder. Er zit een klein verschil op, dat de moeite is --  -- om over na te denken Ik herinner mij dat ik op mijn twaalfde op pauken en percussie begon te spelen. Mijn leraar zei: "Wel, hoe gaan we dat doen? Muziek gaat om luisteren." "Ja, akkoord. En wat is het probleem?" Hij zei: "Hoe ga je dit horen? Hoe ga je dat horen?"' but output 'en zo verder. Er zit een klein verschil op, dat de moeite is -- -- om over na te denken Ik herinner mij dat ik op mijn twaalfde op pauken en percussie begon te spelen. Mijn leraar zei: "Wel, hoe gaan we dat doen? Muziek gaat om luisteren." "Ja, akkoord. En wat is het probleem?" Hij zei: "Hoe ga je dit horen? Hoe ga je dat horen?" Ik zei: "Hoe ga je het horen?"' does not match the prefix. Please check whether the template is correct, or force context/current outputs.


Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  We truly do all understand it, and we have this global digital fireplace, don't we, but I want to 
share you with the world, because you are also a tribe. You are the TED tribe, yeah? But you have to remember that 
compliment.
Input current: You have to stand tall, breathe in through your nose, and I'm going to photograph you. Okay?
Output context: We begrijpen het echt en we hebben zo'n wereldwijd digitaal kampvuur, niet waar? Maar ik wil jullie
met de wereld delen, want jullie zijn ook een stam. Jullie zijn toch de TED-stam? Maar je moet aan dat compliment 
denken.
Output current: Je moet recht staan, door je neus ademen, en ik ga jullie fotograferen. Oké?

#1.
Generated output (CTI > 0.047):  Je moet recht staan, door je neus ademen, en ik ga jullie(0.420) fotograferen. 
Oké?
Input context (CCI > 0.022):     We truly do all understand it, and we have this global digital fireplace, don't 
we, but I want to share you with the world, because you are also a tribe. You are the TED tribe, yeah? But you have
to remember that compliment.
Output context (CCI > 0.024):    We begrijpen het echt en we hebben zo'n wereldwijd digitaal kampvuur, niet waar? 
Maar ik wil jullie(0.069) met de wereld delen, want jullie(0.031) zijn ook een stam. Jul(0.040)lie zijn toch de 
TED-stam? Maar je moet aan dat compliment denken.
#2.
Generated output (CTI > 0.047):  Je moet recht staan, door je neus ademen, en ik ga(0.142) jullie fotograferen. 
Oké?
Input context (CCI > 0.002):     We(0.003) truly do all understand it, and we have this global(0.004) digital 
fireplace, don't we, but I want to(0.002) share you with the world, because you are also a tribe. You are the 
(0.002)TED tribe, yeah? But you have to remember that(0.004) compliment.
Output context (CCI > 0.002):    We begrijpen het echt en we hebben zo'n wereldwijd digitaal kampvuur, niet waar? 
Maar ik wil jullie met de wereld delen, want jullie zijn ook een stam. Jullie zijn toch de TED-stam? Maar je 
moet(0.004) aan dat compliment denken.
#3.
Generated output (CTI > 0.047):  Je moet recht staan(0.119), door je neus ademen, en ik ga jullie fotograferen. 
Oké?
Input context (CCI > 0.003):     We truly do all understand it, and we have this global digital fireplace, don't 
we, but I want to share you with the world, because you are also a tribe. You are the TED tribe, yeah? But you have
to remember that(0.008) compliment.
Output context (CCI > 0.003):    We begrijpen het echt en we hebben zo'n wereldwijd digitaal kampvuur, niet waar? 
Maar ik wil jullie met de wereld delen, want jullie zijn ook een stam. Jullie zijn toch de TED-stam? Maar je moet 
aan dat compliment(0.004) denken(0.004).
#4.
Generated output (CTI > 0.047):  Je(0.107) moet recht staan, door je neus ademen, en ik ga jullie fotograferen. 
Oké?
Input context (CCI > 0.007):     We truly do all understand it, and we have this global digital fireplace, don't 
we, but I want to share you with the world, because you are also a tribe. You are the TED tribe, yeah? But you have
to remember that(0.009) compliment.(0.009)
Output context (CCI > 0.007):    We begrijpen het echt en we hebben zo'n wereldwijd digitaal kampvuur, niet waar? 
Maar ik wil jullie met de wereld delen, want jullie(0.009) zijn ook een stam. Jul(0.020)lie zijn toch de TED-stam? 
Maar je(0.009) moet aan dat compliment denken.(0.010)

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  Sometimes Brian and I walk through the park with Scarlett, and she rolls through the grass, and we 
just look at her and then we look at each other and we feel gratitude. We forget about all of our new middle-class 
frustrations and disappointments, and we feel like millionaires.
Input current: Thank you.
Output context: Soms lopen Brian en ik met Scarlett door het park en rolt ze door het gras. We kijken naar haar en 
dan kijken we naar elkaar en we voelen dankbaarheid. We vergeten onze middenklasse-frustraties en teleurstellingen 
en we voelen ons miljonairs.
Output current: Dankuwel.

#1.
Generated output (CTI > 0.110):  Danku(0.223)wel.
Input context (CCI > 0.007):     Sometimes Brian and I walk through the park with Scarlett, and she rolls through 
the grass, and we just look at her and then we look at each other and we feel gratitude. We forget about all of our
new middle-class frustrations and disappointments, and we feel like millionaires.(0.023)
Output context (CCI > 0.006):    Soms lopen Brian en ik met Scarlett door het park en rolt ze door het gras. We 
kijken naar haar en dan kijken we naar elkaar en we voelen dankbaarheid. We vergeten onze middenklasse-frustraties 
en teleurstellingen en we voelen ons miljon(0.006)airs.(0.010)
#2.
Generated output (CTI > 0.110):  Dank(0.180)uwel.
Input context (CCI > 0.003):     Sometimes Brian and I walk through the park with Scarlett, and she rolls through 
the grass, and we just look at her and then we look at each other and we feel gratitude. We forget about all of our
new middle-class frustrations and disappointments, and we feel like millionaires.(0.013)
Output context (CCI > 0.003):    Soms lopen Brian en ik met Scarlett door het park en rolt ze door het gras. We 
kijken naar haar en dan kijken we naar elkaar en we voelen dankbaarheid. We vergeten onze middenklasse-frustraties 
en teleurstellingen en we voelen ons miljonairs.(0.008)

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  And so he's assembled a group of people. And I've fallen in love with him, as I have with Herbie 
and with his music. And Quincy said, "Where did the idea for centers like this come from?"
Input current: And I said, "It came from your music, man. Because Mr. Ross used to bring in your albums when I was 
16 years old in the pottery class,
Output context: Hij verzamelde een groep mensen. Ik ben helemaal weg van hem, net als van Herbie en zijn muziek. 
Quincy zei: "Waar kwam de idee voor centra als dit vandaan?"
Output current: Ik zei: "Het kwam van jouw muziek, man. Want Mr. Ross bracht je albums mee toen ik 16 jaar oud was 
in de keramiekles.

#1.
Generated output (CTI > 0.066):  Ik zei: "Het(0.364) kwam van jouw muziek, man. Want Mr. Ross bracht je albums mee 
toen ik 16 jaar oud was in de keramiekles.
Input context (CCI > 0.009):     And so he's assembled a group of people. And I've fallen in love with him, as I 
have with Herbie and with his music. And Quincy said, "Where did the idea for centers like this come from?"
Output context (CCI > 0.010):    Hij verzamelde een groep mensen. Ik ben helemaal weg van hem, net als van Herbie 
en zijn muziek. Quincy zei: "Wa(0.024)ar kwam de idee(0.016) voor centra als dit vandaan?"
#2.
Generated output (CTI > 0.066):  Ik zei: "Het kwam van jouw muziek, man. Want Mr. Ross bracht je albums mee toen ik
16 jaar oud was in de keramiekles.(0.282)
Input context (CCI > 0.004):     And so he's assembled a group of people(0.005). And I've fallen in love with him, 
as I have with Herbie and with his music. And(0.006) Quincy said, "Where did the idea for(0.005) centers like this 
come from(0.006)?"
Output context (CCI > 0.004):    Hij verzamelde een(0.005) groep mensen. Ik ben helemaal weg van hem, net als van 
Herbie en zijn muziek. Quincy zei: "(0.007)Waar kwam de idee voor centra als dit vandaan?"(0.006)
#3.
Generated output (CTI > 0.066):  Ik zei: "Het kwam van jouw muziek, man.(0.229) Want Mr. Ross bracht je albums mee 
toen ik 16 jaar oud was in de keramiekles.
Input context (CCI > 0.005):     And so he's assembled a group of people(0.006). And I've fallen in love with him, 
as I have with Herbie and with his music. And(0.008) Quincy said, "Where did the idea for(0.006) centers like this 
come from(0.006)?"
Output context (CCI > 0.005):    Hij verzamelde een groep mensen. Ik ben helemaal weg van hem, net als van Herbie 
en zijn muziek. Quincy zei: "(0.006)Waar kwam de idee voor centra als dit vandaan(0.005)?"
#4.
Generated output (CTI > 0.066):  Ik(0.162) zei: "Het kwam van jouw muziek, man. Want Mr. Ross bracht je albums mee 
toen ik 16 jaar oud was in de keramiekles.
Input context (CCI > 0.005):     And so he's assembled a group of people. And I've fallen in love with him, as I 
have with Herbie and with his music. And(0.007) Quincy said, "Where did the idea for centers like this come 
from?"(0.008)
Output context (CCI > 0.006):    Hij verzamelde een groep mensen. Ik ben helemaal weg van hem, net als van Herbie 
en zijn muziek. Quincy zei: "Waar kwam de idee voor centra als dit vandaan?"(0.015)
#5.
Generated output (CTI > 0.066):  Ik zei: "(0.151)Het kwam van jouw muziek, man. Want Mr. Ross bracht je albums mee 
toen ik 16 jaar oud was in de keramiekles.
Input context (CCI > 0.001):     And so he's assembled a group of people. And I've fallen in love with him, as I 
have with Herbie and with his music. And Quincy said, "Where did the idea for centers like this come from?"
Output context (CCI > 0.001):    Hij verzamelde een groep mensen. Ik ben helemaal weg van hem, net als van Herbie 
en zijn muziek. Quincy zei: "(0.004)Waar kwam de idee voor centra als dit vandaan?"(0.002)
#6.
Generated output (CTI > 0.066):  Ik zei: "Het kwam van jouw muziek, man. Want Mr. Ross bracht je albums mee 


ERROR for 4
Output template '{context} {current}' contains prefix 'LT:  JF: Nee, dat is waar. Ik besta omdat ik vriendinnen heb. Jij bent er één van.' but output 'LT: JF: Nee, dat is waar. Ik besta omdat ik vriendinnen heb. Jij bent er één van. Ik weet niets van jou. Maar hoe dan ook.' does not match the prefix. Please check whether the template is correct, or force context/current outputs.


...

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  One of the weird things that's happened is, after the TED explosion, I got a lot of offers to speak
all over the country -- everyone from schools and parent meetings to Fortune 500 companies. And so many of the 
calls went like this, "Dr. Brown, we loved your TED talk. We'd like you to come in and speak.
Input current: We'd appreciate it if you wouldn't mention vulnerability or shame."
Output context: Eén van de rare dingen die gebeurden na de TED-explosie, is dat ik overal gevraagd werd om te 
spreken -- van scholen en ouderavonden tot Fortune 500-bedrijven. Vele gesprekken verliepen als volgt: "Hallo, Dr. 
Brown. We zijn dol op uw TEDTalk. We willen u graag als spreker.
Output current: We zouden het waarderen als u niet sprak over kwetsbaarheid of schaamte."

#1.
Generated output (CTI > 0.196):  We zouden het waarderen als u(2.811) niet sprak over kwetsbaarheid of schaamte."
Input context (CCI > 0.010):     One of the weird things that's happened is, after the TED explosion, I got a lot 
of offers to speak all over the country -- everyone from schools and parent meetings to Fortune 500 companies. And 
so many of the calls went like this, "Dr. Brown, we loved your TED talk. We'd like you to come in and speak.
Output context (CCI > 0.012):    Eén van de rare dingen die gebeurden na de TED-explosie, is dat ik overal gevraagd
werd om te spreken -- van scholen en ouderavonden tot Fortune 500-bedrijven. Vele gesprekken verliepen als volgt: 
"Hallo, Dr. Brown. We zijn dol op uw TEDTalk. We willen u(0.058) graag als spreker.
#2.
Generated output (CTI > 0.196):  We zouden het waarderen als u niet (0.414)sprak over kwetsbaarheid of schaamte."
Input context (CCI > 0.000):     One(0.000) of(0.000) the(0.000) weird(0.000) things(0.000) that's happened is, 
after the TED explosion, I got a lot of offers to speak all over the country -- everyone from schools and parent 
meetings to Fortune 500 companies. And so many of the calls went like this, "Dr. Brown, we loved your TED talk. 
We'd like you to come in and speak.
Output context (CCI > 0.000):    E(0.000)én(0.000) van(0.000) de(0.000) rare(0.000) dingen die gebeurden na de 
TED-explosie, is dat ik overal gevraagd werd om te spreken -- van scholen en ouderavonden tot Fortune 
500-bedrijven. Vele gesprekken verliepen als volgt: "Hallo, Dr. Brown. We zijn dol op uw TEDTalk. We willen u graag
als spreker.
#3.
Generated output (CTI > 0.196):  We zouden het waarde(0.224)ren als u niet sprak over kwetsbaarheid of schaamte."
Input context (CCI > 0.002):     One of the(0.003) weird things that's happened is, after the TED explosion, I got 
a lot of offers to speak all over the country -- everyone from schools and parent meetings to(0.004) Fortune 500 
companies. And so many of the calls went like this, "(0.003)Dr. Brown, we loved your TED talk. We'd like you to 
come in and speak(0.003).(0.004)
Output context (CCI > 0.002):    Eén van de rare dingen die gebeurden na de TED-explosie, is dat ik overal gevraagd
werd om te spreken -- van scholen en ouderavonden tot Fortune 500-bedrijven. Vele gesprekken verliepen als volgt: 
"Hallo, Dr. Brown. We zijn dol op uw TEDTalk. We willen u graag(0.003) als(0.004) spre(0.002)ker.(0.003)

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  And after the trial had been running for a little while, he gathered together all his colleagues 
around his table, and he said, "Well, gentlemen, we have some preliminary results. They're not statistically 
significant. But we have something.
Input current: And it turns out that you're right and I'm wrong.
Output context: Nadat de studie een tijdje bezig was, verzamelde hij al zijn collega's rond zijn tafel, en hij zei:
“Nou, heren, we hebben een aantal voorlopige resultaten. Ze zijn niet statistisch significant. Maar we hebben iets.
Output current: En het blijkt dat jullie gelijk hebben en ik verkeerd.

#1.
Generated output (CTI > 0.101):  En het blijkt dat jullie(0.630) gelijk hebben en ik verkeerd.
Input context (CCI > 0.005):     And after the trial had been running for a little while, he gathered together all 
his colleagues around his table, and he said, "Well,(0.014) gent(0.005)le(0.009)men, we have some preliminary 
results. They're not statistically significant. But we have something.
Output context (CCI > 0.004):    Nadat de studie een tijdje bezig was, verzamelde hij al zijn collega's rond zijn 
tafel, en hij zei: “Nou, heren, we hebben een aantal voorlopige resultaten. Ze zijn niet statistisch significant. 
Maar we hebben iets.
#2.
Generated output (CTI > 0.101):  En(0.183) het blijkt dat jullie gelijk hebben en ik verkeerd.
Input context (CCI > 0.008):     And after the trial had been running for a little while, he gathered together all 
his colleagues around his table, and he said, "Well, gentlemen, we have some preliminary results. They're not 
statistically significant. But we have something(0.009).(0.015)
Output context (CCI > 0.009):    Nadat de studie een tijdje bezig was, verzamelde hij al zijn collega's rond zijn 
tafel, en hij zei: “Nou, heren, we hebben een aantal voorlopige resultaten. Ze zijn niet statistisch significant. 
Maar we hebben iets.(0.026)
#3.
Generated output (CTI > 0.101):  En het blijkt dat jullie gelijk hebben en ik verkeerd(0.137).
Input context (CCI > 0.001):     And after the trial had been running for a little while, he gathered together all 
his(0.001) colleagues around his table, and he said, "Well, gentlemen, we have some(0.001) preliminary results. 
They're not(0.002) statistically(0.002) significant. But we have(0.002) something.
Output context (CCI > 0.001):    Nadat de studie een tijdje bezig was, verzamelde hij al zijn collega's rond zijn 
tafel, en hij zei: “Nou, heren, we hebben een aantal voorlopig(0.002)e resultaten. Ze zijn niet statistisch 
significant. Maar we hebben iets(0.002).

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  there's one more thing, there's one final piece of the puzzle. Gwen, I am so grateful for your 
choices
Input current: because if we take a look at the first letters of your combinations, we get "C-H-A-O-S" for chaos 
and "O-R-D-E-R" for order.
Output context: Nog één ding. Het laatste puzzelstukje. Gwen, ik ben zo dankbaar voor jouw keuzes.
Output current: Als we kijken naar de eerste letters van je combinaties, krijgen we "C-H-A-O-S" voor chaos en 
"O-R-D-E-R" voor orde.

#1.
Generated output (CTI > 0.021):  Als we kijken naar de eerste letters van je(0.451) combinaties, krijgen we 
"C-H-A-O-S" voor chaos en "O-R-D-E-R" voor orde.
Input context (CCI > 0.011):     there's one more thing, there's one final piece of the puzzle. Gwen, I am so 
grateful for your choices
Output context (CCI > 0.012):    Nog één ding. Het laatste puzzelstukje. Gwen, ik ben zo dankbaar voor jouw(0.032) 
keuzes.
#2.
Generated output (CTI > 0.021):  Als(0.309) we kijken naar de eerste letters van je combinaties, krijgen we 
"C-H-A-O-S" voor chaos en "O-R-D-E-R" voor orde.
Input context (CCI > 0.023):     there's one more thing, there's one final piece of the puzzle.(0.040) Gwen, I am 
so grateful for your choices(0.025)
Output context (CCI > 0.024):    Nog één ding. Het laatste puzzelstukje. Gwen, ik ben zo dankbaar voor jouw 
keuzes.(0.043)
#3.
Generated output (CTI > 0.021):  Als we kijken(0.024) naar de eerste letters van je combinaties, krijgen we 
"C-H-A-O-S" voor chaos en "O-R-D-E-R" voor orde.
Input context (CCI > 0.005):     there's one more thing, there's one final piece of the puzzle.(0.010) Gwen, I am 
so(0.005) grateful for your choices
Output context (CCI > 0.005):    Nog één ding. Het laatste puzzelstukje. Gwen, ik ben zo dankbaar voor jouw(0.005) 
keuzes.

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  We do that in our personal lives. We do that corporate -- whether it's a bailout, an oil spill ... 
a recall. We pretend like what we're doing doesn't have a huge impact on other people. I would say to companies, 
this is not our first rodeo, people.
Input current: We just need you to be authentic and real and say ... "We're sorry. We'll fix it." But there's 
another way, and I'll leave you with this.
Output context: Dat doen we in ons persoonlijke leven. We doen dat in het bedrijfsleven -- of het nu gaat om steun 
aan banken of een olieramp een terugroepactie -- we doen net alsof wat we doen geen enorme impact heeft op andere 
mensen. Ik wil tegen bedrijven zeggen: "We zijn niet van gisteren, mensen.
Output current: We moeten authentiek en echt zijn en zeggen: "Het spijt ons. We zullen het repareren." Maar er is 
nog een andere manier, en ik zal jullie hiermee achterlaten.

#1.
Generated output (CTI > 0.056):  We moeten authentiek en echt zijn en zeggen: "Het(0.402) spijt ons. We zullen het 
repareren." Maar er is nog een andere manier, en ik zal jullie hiermee achterlaten.
Input context (CCI > 0.002):     We do that in our personal lives. We do that corporate -- whether it's a bailout, 
an oil spill ... a recall. We pretend like what we're doing doesn't have a huge impact on other people. I would say
to companies, this is not our first(0.003) rodeo, people.
Output context (CCI > 0.002):    Dat doen we in ons persoonlijke leven. We doen dat in het bedrijfsleven -- of het 
nu gaat om steun aan banken of een olieramp een terugroepactie -- we doen net alsof wat we doen geen enorme impact 
heeft op andere mensen. Ik wil tegen bedrijven zeggen: "(0.003)We(0.004) zijn niet van gi(0.005)ster(0.004)en, 
mensen.
#2.
Generated output (CTI > 0.056):  We moeten aut(0.371)hentiek en echt zijn en zeggen: "Het spijt ons. We zullen het 
repareren." Maar er is nog een andere manier, en ik zal jullie hiermee achterlaten.
Input context (CCI > 0.008):     We do that in our personal lives. We do that corporate -- whether it's a bailout, 
an oil spill ... a recall. We pretend like what we're doing doesn't have a huge impact on other people. I would say
to companies, this is not our first(0.011) rodeo,(0.010) people(0.009).(0.014)
Output context (CCI > 0.008):    Dat doen we in ons persoonlijke leven. We doen dat in het bedrijfsleven -- of het 
nu gaat om steun aan banken of een olieramp een terugroepactie -- we doen net alsof wat we doen geen enorme impact 
heeft op andere mensen. Ik wil tegen bedrijven zeggen: "(0.009)We zijn niet van gisteren, mensen(0.013).(0.013)
#3.
Generated output (CTI > 0.056):  We(0.362) moeten authentiek en echt zijn en zeggen: "Het spijt ons. We zullen het 
repareren." Maar er is nog een andere manier, en ik zal jullie hiermee achterlaten.
Input context (CCI > 0.003):     We do that in our personal lives. We do that corporate -- whether it's a bailout, 
an oil spill ... a recall. We pretend like what we're doing doesn't have a huge impact on other people. I would say
to companies, this is not our first rodeo, people.(0.006)
Output context (CCI > 0.003):    Dat doen we in ons persoonlijke leven. We doen dat in het bedrijfsleven -- of het 
nu gaat om steun aan banken of een olieramp een terugroepactie -- we doen net alsof wat we doen geen enorme impact 
heeft op andere mensen. Ik wil tegen bedrijven zeggen: "We zijn niet van gisteren,(0.003) mensen(0.004).(0.008)
#4.
Generated output (CTI > 0.056):  We moeten(0.099) authentiek en echt zijn en zeggen: "Het spijt ons. We zullen het 
repareren." Maar er is nog een andere manier, en ik zal jullie hiermee achterlaten.
Input context (CCI > 0.006):     We do that in our personal lives. We do that corporate -- whether it's a bailout, 
an oil spill ... a recall. We pretend l

Context with contextual cues (std λ=2.00, top 5) followed by output sentence with context-sensitive target spans 
(all)
(CTI = "kl_divergence", CCI = "saliency" w/ "contrast_prob_diff" target)

Input context:  The rabbi came in and saw what was going on. And he called the two of them to his office. And he 
said, you know, "This is what's happening."
Input current: And the rich man -- oh, dear -- crestfallen. "You mean God didn't want my bread?"
Output context: De rabbijn zag wat er aan de hand was. En hij riep de beide mannen in zijn kantoor. En hij zei: 
"Weten jullie, dit is er aan de hand."
Output current: En de rijke man -- oh, lieverd -- sloeg op. "Bedoel je dat God mijn brood niet wilde?"

#1.
Generated output (CTI > 0.046):  En de rijke man -- oh, lieverd -- sloeg op.(0.350) "Bedoel je dat God mijn brood 
niet wilde?"
Input context (CCI > 0.005):     The(0.010) rabbi came in and saw what was going on. And he called the two of them 
to his office. And he said, you know, "This is what's(0.006) happening."
Output context (CCI > 0.004):    De rabbijn zag wat er aan de hand was. En hij riep(0.005) de beide mannen in zijn 
kantoor. En hij zei: "Weten jullie, dit is er aan de hand."
#2.
Generated output (CTI > 0.046):  En de rijke man -- oh, lieverd -- slo(0.194)eg op. "Bedoel je dat God mijn brood 
niet wilde?"
Input context (CCI > 0.001):     The(0.002) rabbi came in and saw what was going on. And he called the two of them 
to his office. And he said, you know, "This is what's(0.001) happening."
Output context (CCI > 0.001):    De rabbijn zag wat er aan de hand was. En hij riep(0.001) de beide mannen in zijn 
kantoor. En hij zei: "Weten jullie, dit is er aan de hand."
#3.
Generated output (CTI > 0.046):  En(0.154) de rijke man -- oh, lieverd -- sloeg op. "Bedoel je dat God mijn brood 
niet wilde?"
Input context (CCI > 0.016):     The rabbi came in and saw what was going on. And he called the two of them to his 
office. And he said, you know, "This is what's happening."
Output context (CCI > 0.018):    De rabbijn zag wat er aan de hand was. En(0.022) hij riep de beide mannen in zijn 
kantoor. En hij zei: "Weten jullie, dit is er aan de hand."(0.044)
#4.
Generated output (CTI > 0.046):  En de rijke man -- oh, lieverd -- sloeg op(0.093). "Bedoel je dat God mijn brood 
niet wilde?"
Input context (CCI > 0.002):     The(0.005) rabbi came in and saw what was going on. And he called the two of them 
to his office. And he said, you know, "This is what's(0.003) happening."
Output context (CCI > 0.002):    De rabbijn zag wat er aan de hand was. En hij riep de beide mannen in zijn 
kantoor. En hij zei: "Weten jullie, dit is er aan de hand."
#5.
Generated output (CTI > 0.046):  En de rijke man -- oh, lieverd -- sloeg op. "(0.071)Bedoel je dat God mijn brood 
niet wilde?"
Input context (CCI > 0.002):     The rabbi came in and saw what was going on. And he called the two of them to his 
office. And he said, you know, "This is what's happening."
Output context (CCI > 0.002):    De rabbijn zag wat er aan de hand was. En hij riep de beide mannen in zijn 
kantoor. En hij zei: "(0.006)Weten jullie, dit is er aan de hand."(0.003)
#6.
Generated output (CTI > 0.046):  En de rijke man -- oh, liever(0.065)d -- sloeg op. "Bedoel je dat God mijn brood 
niet wilde?"
Input context (CCI > 0.003):     The(0.004) rabbi came in and saw what was going on. And he called the two of them 
to his office. And he said, you know, "This is what's happening."
Output context (CCI > 0.003):    De rabbijn zag wat er aan de hand was. En hij riep de beide mannen in zijn 
kantoor. En hij zei: "We(0.003)ten(0.003) jullie(0.005), dit is er aan de hand."
#7.
Generated output (CTI > 0.046):  En de rijke man -- oh, lieverd -- sloeg op. "Be(0.061)doel je dat God mijn brood 
niet wilde?"
Input context (CCI > 0.001):     The(0.002) rabbi came in and saw what was going on. And he called the two of them 
to his office. And he said, you know, "This is what's happening."
Output contex